In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..')+'/src')
if module_path not in sys.path:
    sys.path.append(module_path)

: 

# Ansatz

In this file, we illustrate the use of the Class ansatz.

In [10]:
import torch
from torch import nn
import warnings
import config

In [ ]:
import warnings
import torch
import torch as tc
from torch.func import jacfwd, vmap, grad,jacrev

class Ansatz(tc.nn.Module):
    
    def __init__(self, d, m, theta=None, kind='gaussian'):
        
        super().__init__()
        self.m = m       # model parameter 
        self.d = d       # dimension of b (same as x)
        
        # size of theta according to the the model
        if kind == 'gaussian':
            self.s = 2*m+d*m 
        elif kind== 'nn':
            self.L = 0.5
            self.l = 3
            self.s = m + m*d * self.l + d * self.l 
        else:
            self.s = 2*m+d*m
            warnings.warn("The type of network has not The theta parameter has not been initialized") 
        
        # Dictionary of models 
        methods = {
            'gaussian'        : self.gaussian_x,   # Eq. (16)
        }

        # Warning : une initialisation à ones fait échouer la minimisation pour theta0 !
        self.Theta = tc.rand(self.s,requires_grad=True) #if theta is None else theta
        self.theta = tc.nn.Parameter(tc.rand(self.s) if theta is None else theta)
         #self.theta = tc.nn.Parameter(2.0*tc.ones(self.s) if theta is None else theta) 

        # batched derivatives , input are (d,)
        self.eval_x    = methods[kind]
        self.forward   = vmap(self.eval_x)
        self.dx        = vmap(grad(self.eval_x),             in_dims=(0,) )
        self.dxx       = vmap(grad(grad(self.eval_x)),       in_dims=(0,) )
        self.dxxx      = vmap(grad(grad(grad(self.eval_x))), in_dims=(0,) )

        # work in progress
        # self.dtheta = vmap(lambda x : autograd(self.eval_x(x), self.theta)[0])

        
    def splitter_gaussian(self):
        c = self.theta[:self.m]
        b = self.theta[self.m: self.m+self.m*self.d ].reshape(self.m,self.d)
        w = self.theta[self.m+self.m*self.d:]
        return (c,b,w)
    
    def gaussian_x(self, x: tc.Tensor):
               
       # Input :  a SCALAR x of shape (0,)
       # Output : a SCALAR
        
        c, b, w = self.splitter_gaussian()
        return tc.sum(c*tc.exp(-w**2 *(x-b).T**2))  

    
    """
    
    def forward_t(self, theta, x):
        self.theta = theta
        c,b,w = self.splitter_gaussian()
        diff = x.unsqueeze(2)-b.t().unsqueeze(0) # tensor with shape (npoints,d,m)
        distance = torch.norm(diff, dim=1)**2 # tensor of dimension (npoints, m)
        exp_factor = torch.exp(-w.unsqueeze(0)**2*distance)
        return torch.sum(c.unsqueeze(0)*exp_factor, dim=1).unsqueeze(1) """
    def forward_t(self, theta, x):
        self.Theta = theta.clone()
        c,b,w = self.splitter_gaussian1()
        #vect=lambda x: (c*tc.exp(-w**2*(x-b)**2)).sum()
        #return (c*tc.exp(-w**2*(x-b)**2)).sum()
        return tc.sum(c*tc.exp(-w**2*(x-b).T**2))
    def grad_theta(self, theta, samples):
        self.Theta = theta.clone()
        jacobian=jacrev(self.forward_t,argnums=0)(self.Theta, samples)
        return jacobian
        
    def out_prod(self,theta,x):
        #self.Theta=theta.clone()
        out_p=lambda theta,x_: tc.outer(self.grad_theta(theta,x_),self.grad_theta(theta,x_))
        M=vmap((out_p),in_dims=(None,0))(theta,x).detach()
        return tc.mean(M,dim=0)
    
    def __call__(self, x):
        return self.forward(x)
        

: 

In [12]:
d = 2
m = 10
model = Ansatz(d,m)
x_train = torch.rand(10, 2)
print(x_train)
print(model(x_train))

tensor([[0.3597, 0.2799],
        [0.9195, 0.7244],
        [0.7365, 0.8386],
        [0.0975, 0.9521],
        [0.5218, 0.2038],
        [0.7214, 0.0835],
        [0.3705, 0.5307],
        [0.0427, 0.8269],
        [0.6966, 0.8319],
        [0.6958, 0.7609]])


/tmp/ipykernel_55293/3572027452.py:6: UserWarning: The neural network has been initiated with random theta
  warnings.warn("The neural network has been initiated with random theta")


RuntimeError: The size of tensor a (2) must match the size of tensor b (10) at non-singleton dimension 1